In [1]:
import os
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, CSVLogger, EarlyStopping
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import tensorflow as tf

In [2]:
from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)


Mounted at /content/gdrive


In [3]:
%cd /content/gdrive/MyDrive/

/content/gdrive/MyDrive


In [5]:
class_names = ['Basale',
               'Guava',
               'Neem',
               'Sandalwood',
               'Tulsi']
# class_names_label = {class_name:i for i, class_name in enumerate(class_names)}

# nb_classes = len(class_names)

# IMAGE_SIZE = (150, 150)

In [6]:
trainDataGen = ImageDataGenerator(rotation_range=5,
                                  width_shift_range=0.1,
                                  height_shift_range=0.1,
                                  rescale=1.0 / 255,
                                  shear_range=0.2,
                                  zoom_range=0.2,
                                  horizontal_flip=False,
                                  fill_mode='nearest')

testDataGen = ImageDataGenerator(rescale=1.0 / 255)

trainGenerator = trainDataGen.flow_from_directory(os.path.join('Dataset', 'Train'),
                                                  target_size=(150, 150),
                                                  batch_size=32,
                                                  color_mode='rgb',
                                                  classes=[str(Class)
                                                           for Class in range(5)],
                                                  class_mode='categorical')

validationGenerator = testDataGen.flow_from_directory(os.path.join('Dataset', 'Validation'),
                                                      target_size=(150, 150),
                                                      batch_size=32,
                                                      color_mode='rgb',
                                                      classes=[
                                                          str(Class) for Class in range(5)],
                                                      class_mode='categorical')

Found 303 images belonging to 5 classes.
Found 35 images belonging to 5 classes.


In [7]:
model = Sequential()
model.add(Conv2D(32, (5, 5), padding = 'same', activation = 'relu', kernel_initializer = 'he_uniform', input_shape = (150, 150, 3)))
model.add(Conv2D(32, (5, 5), padding = 'same', activation = 'relu', kernel_initializer = 'he_uniform'))
model.add(Conv2D(32, (5, 5), padding = 'same', activation = 'relu', kernel_initializer = 'he_uniform'))
model.add(MaxPooling2D((2, 2), strides = (2, 2)))

model.add(Conv2D(64, (3, 3), padding = 'same', activation = 'relu', kernel_initializer = 'he_uniform'))
model.add(Conv2D(64, (3, 3), padding = 'same', activation = 'relu', kernel_initializer = 'he_uniform'))
model.add(Conv2D(64, (3, 3), padding = 'same', activation = 'relu', kernel_initializer = 'he_uniform'))
model.add(MaxPooling2D((2, 2), strides = (2, 2)))

model.add(Flatten())

model.add(Dense(256, activation = 'relu', kernel_initializer = 'he_uniform'))
model.add(Dropout(0.2))

model.add(Dense(5, activation = 'softmax'))

model.compile(optimizer = Adam(learning_rate = 1e-3, decay = 1e-5), loss = 'categorical_crossentropy', metrics = ['accuracy'])


In [8]:

if not os.path.isdir('Model_2'):
    os.mkdir('Model_2')

callbacks = [ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                               patience=7, min_lr=1e-5),
             EarlyStopping(monitor='val_loss', patience=9,  # Patience should be larger than the one in ReduceLROnPlateau
                           min_delta=1e-5),
             CSVLogger(os.path.join('Model_2', 'training.log'), append=True),
             ModelCheckpoint(os.path.join(
                 'Model_2', 'backup_last_model.hdf5')),
             ModelCheckpoint(os.path.join('Model_2', 'best_val_acc.hdf5'),
                             monitor='val_accuracy', mode='max', save_best_only=True),
             ModelCheckpoint(os.path.join('Model_2', 'best_val_loss.hdf5'), monitor='val_loss', mode='min', save_best_only=True)]

model.fit(trainGenerator, epochs=20,
          validation_data=validationGenerator, callbacks=callbacks)


Epoch 1/20
10/10 [==============================] - 127s 13s/step - loss: 11.2362 - accuracy: 0.2244 - val_loss: 1.7033 - val_accuracy: 0.3429 - lr: 0.0010
Epoch 2/20
10/10 [==============================] - 114s 11s/step - loss: 1.5311 - accuracy: 0.3663 - val_loss: 1.4301 - val_accuracy: 0.4286 - lr: 0.0010
Epoch 3/20
10/10 [==============================] - 115s 11s/step - loss: 1.4271 - accuracy: 0.3861 - val_loss: 1.0489 - val_accuracy: 0.6857 - lr: 0.0010
Epoch 4/20
10/10 [==============================] - 106s 11s/step - loss: 1.0955 - accuracy: 0.5017 - val_loss: 0.9562 - val_accuracy: 0.6571 - lr: 0.0010
Epoch 5/20
10/10 [==============================] - 113s 11s/step - loss: 0.9707 - accuracy: 0.6139 - val_loss: 0.6821 - val_accuracy: 0.7714 - lr: 0.0010
Epoch 6/20
10/10 [==============================] - 109s 11s/step - loss: 0.8175 - accuracy: 0.6898 - val_loss: 0.5401 - val_accuracy: 0.7429 - lr: 0.0010
Epoch 7/20
10/10 [==============================] - 108s 11s/step - l

In [9]:
model = load_model(os.path.join('Model_2', 'best_val_loss.hdf5'))
loss, acc = model.evaluate(validationGenerator)
print('Loss on Validation Data : ', loss)
print('Accuracy on Validation Data :', '{:.4%}'.format(acc))  

2/2 [==============================] - 3s 211ms/step - loss: 0.2567 - accuracy: 0.9714
Loss on Validation Data :  0.2566501498222351
Accuracy on Validation Data : 97.1429%


In [10]:
import numpy as np
import PIL.ImageOps as ImageOps
import PIL.Image as Image
import matplotlib.pyplot as plt

In [11]:
data = np.ndarray(shape=(1, 150, 150, 3), dtype=np.float32)

class_names = ['Basale',
               'Guava',
               'Neem',
               'Sandalwood',
               'Tulsi']
image = Image.open('/content/gdrive/MyDrive/plant_dataset/Neem/AI-S-008.jpg')
#image sizing
# image = ImageOps.grayscale(image)

size = (150, 150)
image = ImageOps.fit(image, size, Image.ANTIALIAS)

#turn the image into a numpy array
image_array = np.asarray(image)
# np.reshape(image_array, newshape=(150, 150,3))
# Normalize the image
normalized_image_array = (image_array.astype(np.float32) / 127.0) - 1
print(normalized_image_array.shape)
# Load the image into the array
data[0] = normalized_image_array

# run the inference
prediction = model.predict(data)
pred_label = np.argmax(prediction, axis = 1) # We take the highest probability
class_prediction = class_names[pred_label[0]]

(150, 150, 3)
1/1 [==============================] - 0s 265ms/step


In [12]:

if class_prediction == 'Basale':
    print('Basale: Basale has an anti-inflammatory activity and wound healing ability. It can be helpful as a first aid, and the leaves of this plant can be crushed and applied to burns, scalds and wounds to help in healing of the wounds.')
if class_prediction == 'Guava':
    print('Guava: Aside from bearing a delicious taste, the fruit of the Guava tree is a rich source of Vitamin C and antioxidants. It is especially effective against preventing infections such as Gastrointestinal infections, Respiratory infections, Oral/dental infections and Skin infections. It can also aid in the treatment of Hypertension, Fever, Pain, Liver and Kidney problems. ')
if class_prediction == 'Neem':
    print('Neem: Prevalent in traditional remedies from a long time, Neem is considered as a boon for Mankind. It helps to cure many skin diseases such as Acne, fungal infections, dandruff, leprosy, and also nourishes and detoxifies the skin. It also boosts your immunity and act as an Insect and Mosquito Repellent. It helps to reduce joint paint as well and prevents Gastrointestinal Diseases')
if class_prediction == 'Sandalwood':
    print('Sandalwood: Sandalwood is used for treating the common cold, cough, bronchitis, fever, and sore mouth and throat. It is also used to treat urinary tract infections (UTIs), liver disease, gallbladder problems, heatstroke, gonorrhea, headache, and conditions of the heart and blood vessels (cardiovascular disease).')
if class_prediction == 'Tulsi':
    print('Tulsi: Tulsi plant has the potential to cure a lot of ailments, and is used a lot in traditional remedies. Tulsi can help cure fever, to treat skin problems like acne, blackheads and premature ageing, to treat insect bites. Tulsi is also used to treat heart disease and fever, and respiratory problems.')


Neem: Prevalent in traditional remedies from a long time, Neem is considered as a boon for Mankind. It helps to cure many skin diseases such as Acne, fungal infections, dandruff, leprosy, and also nourishes and detoxifies the skin. It also boosts your immunity and act as an Insect and Mosquito Repellent. It helps to reduce joint paint as well and prevents Gastrointestinal Diseases
